Nicholas Brower<br>Springboard DSCT - May 2022


# Unit 8.3.3 SQL Case Study - Country Club (Tier 2)

This notebook contains my answers to all questions presented in the "SQLTasks Tier 2.sql" file included with this project. As instructed, the queries I wrote to answer questions 1 through 9 were pasted in that file under each question of the same number. <br><br>The same queries are assigned to variables a_1 to a_9 in code cells of this notebook. Their results sets, as generated by a local instance of the same database, are assigned to q_1 to q_9.<br>

<a id = 'files'></a>**Files** <br><br>
The MySQL export file I downloaded from `sql.springboard.com` can be found at [data/raw/country-club-db.sql](data/raw/country-club-db.sql). <br>The local build of the country club database can be found at [data/sqlite_db/country_club.db](data/sqlite_db/country_club.db) <br>The SQLTasks Tier 2.sql file can be located at [SQLTasks Tier 2.sql](SQLTasks%20Tier%202.sql)
***

In [1]:
from contextlib import closing
from textwrap import fill
import re
import sqlite3


In [2]:
def setup_db(
        db_path: str = 'data/sqlite_db/country_club.db',
        sql_path: str = 'data/raw/country-club-db.sql') -> None:
    '''Create a sqlite db file at the path specified by db_path, using a SQL
    dump file at sql_path.
    '''
    db_path = 'file:' + db_path + '?mode=rwc'
    with open(sql_path, 'r') as sql_file_obj:
        with closing(sqlite3.connect(db_path, uri=True)) as con:
            with closing(con.cursor()) as cur:
                sql_dump = sql_file_obj.read()
                commands = re.findall(r'[^;]*;', sql_dump)
                for command in commands:
                    cur.execute(command)
                con.commit()

def query_print(
        columns: tuple[str], rows: tuple[tuple[object]],
        row_nums: bool = True, pg_w: int = 79) -> None:
    '''Print the result of a query formatted with dividers and space-padded
    cells.
    
    Arguments:
    columns -- column names for a result set as a sequence of strings
    rows -- records of a result set, with rows[i][j] the value for 
            the row at rows[i] for the column at columns[j]
    row_nums -- if True, print 0-indexed row number for each record
    pg_w -- the number of underscores to print as a line divider
    
    '''
    if not rows:
        # set empty row number header. set column width
        r_num_header = ''
        col_len = {columns[i]: len(columns[i]) for i in range(len(columns))}
    else:
        # set row number width
        r_num_len = len(f'{len(rows)}')
        #set row number header
        r_num_header = '' + row_nums * f'{"#".rjust(r_num_len)}  |  '
        # set row number cell format
        row_nums = [
            row_nums * f'{str(n).rjust(r_num_len)}  |  '
            for n in range(len(rows))
        ]
        # set column width to longest string in column name, column values
        col_len = {
            columns[i] : max([
                len(columns[i]),
                max([len(f'{row[i]}') for row in rows])
            ])
            for i in range(len(columns))
    }
    # print column headers
    print('')
    print(r_num_header  + '    |  '.join(
        col.replace('_', ' ').ljust(col_len[col]) for col in columns
    ))
    print(pg_w * '_')
    # if result is empty, set number of rows to 0. else print rows
    if not rows:
        row_c = 0
    else:
        for n in range(len(rows)):
            row = rows[n]
            print(row_nums[n] + '    |  '.join(
                f'{row[i]}'.ljust(col_len[columns[i]])
                for i in range(len(columns))
            ))
        row_c = len(rows)
    print(pg_w * '_')
    # print number of rows and number of columns in result
    print(f'result: {row_c} rows, {len(columns)} columns\n\n')

def query_result(
        query:str, 
        path_to_db: str = 'file:data/sqlite_db/country_club.db?mode=ro',
        print_result: bool = True,
        row_nums: bool = True,
        pg_w: int = 79
    ) -> tuple[tuple[str], tuple[object]]:
    '''Open a connection to a SQL database. Return the result of a query.
    
    Arguments:
    query -- a string containing a sqlite query
    path_to_db -- path to the database to query
    print_result -- if True, print the result generated by the query
    row_nums -- if True, print row numbers
    pg_w -- line width for horizontal rule
    
    Return a nested tuple of (columns, rows), with columns a tuple of 
    column names and rows a tuple of rows in the result set, where 
    row[i] is the value of a row for columns[i] for row in rows.
    
    '''
    with closing(sqlite3.connect(path_to_db, uri = True)) as con:
        with closing(con.cursor()) as cur:
            cur.execute(query)
            columns = tuple(_[0] for _ in cur.description)
            rows = tuple(cur.fetchall())
    if print_result:
        query_print(columns, rows, row_nums, pg_w)
    return (columns, rows)


Create a local db file using a MySQL export from https://sql.springboard.com/

In [3]:
setup_db()

### Questions 1-9

<a id = 'q1'></a>
**Q 1**

In [4]:
a_1 = '''
SELECT name
FROM Facilities
WHERE membercost != 0
;
'''
print(f'\n\nfacilities that charge a fee to members')
q_1 = query_result(a_1)



facilities that charge a fee to members

#  |  name          
_______________________________________________________________________________
0  |  Tennis Court 1
1  |  Tennis Court 2
2  |  Massage Room 1
3  |  Massage Room 2
4  |  Squash Court  
_______________________________________________________________________________
result: 5 rows, 1 columns




<a id = 'q2'></a>
**Q 2**


In [5]:
a_2 = '''
SELECT COUNT(facid) AS number_of_facilities
FROM Facilities
WHERE membercost = 0
;
'''
print('\n\nfacilities that are free for members')
q_2 = query_result(a_2, row_nums = False)



facilities that are free for members

number of facilities
_______________________________________________________________________________
4                   
_______________________________________________________________________________
result: 1 rows, 1 columns




<a id = 'q3'></a>**Q 3**

In [6]:
a_3 = '''
SELECT name, facid, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < 0.2 * monthlymaintenance
;
'''
print('\n\nfacilities with member costs less than 20% of monthly maintenance\n\n')
q_3 = query_result(a_3)



facilities with member costs less than 20% of monthly maintenance



#  |  name               |  facid    |  membercost    |  monthlymaintenance
_______________________________________________________________________________
0  |  Tennis Court 1     |  0        |  5             |  200               
1  |  Tennis Court 2     |  1        |  5             |  200               
2  |  Badminton Court    |  2        |  0             |  50                
3  |  Table Tennis       |  3        |  0             |  10                
4  |  Massage Room 1     |  4        |  9.9           |  3000              
5  |  Massage Room 2     |  5        |  9.9           |  3000              
6  |  Squash Court       |  6        |  3.5           |  80                
7  |  Snooker Table      |  7        |  0             |  15                
8  |  Pool Table         |  8        |  0             |  15                
_______________________________________________________________________________
result: 9

<a id = 'q4'></a>**Q 4**

In [7]:
a_4 = '''
SELECT *
FROM Facilities
WHERE facid IN (1, 5)
;
'''
print('\n\nfacilities with facid in (1, 5)')
q_4 = query_result(a_4, pg_w = 112)




facilities with facid in (1, 5)

#  |  facid    |  name              |  membercost    |  guestcost    |  initialoutlay    |  monthlymaintenance
________________________________________________________________________________________________________________
0  |  1        |  Tennis Court 2    |  5             |  25           |  8000             |  200               
1  |  5        |  Massage Room 2    |  9.9           |  80           |  4000             |  3000              
________________________________________________________________________________________________________________
result: 2 rows, 6 columns




<a id = 'q5'></a>**Q 5**

In [8]:
a_5 = '''
SELECT
    name AS facility_name,
    monthlymaintenance AS monthly_maintenance_cost,
    CASE
        WHEN monthlymaintenance > 100 THEN 'expensive'
        ELSE 'cheap'
    END AS maintenance_cost_label
FROM 
    Facilities
ORDER BY
    monthlymaintenance
;
'''
print('\n\nfacilities by monthly maintenance cost')
q_5 = query_result(a_5)




facilities by monthly maintenance cost

#  |  facility name      |  monthly maintenance cost    |  maintenance cost label
_______________________________________________________________________________
0  |  Table Tennis       |  10                          |  cheap                 
1  |  Snooker Table      |  15                          |  cheap                 
2  |  Pool Table         |  15                          |  cheap                 
3  |  Badminton Court    |  50                          |  cheap                 
4  |  Squash Court       |  80                          |  cheap                 
5  |  Tennis Court 1     |  200                         |  expensive             
6  |  Tennis Court 2     |  200                         |  expensive             
7  |  Massage Room 1     |  3000                        |  expensive             
8  |  Massage Room 2     |  3000                        |  expensive             
__________________________________________________________

<a id = 'q6'></a>**Q 6**

In [9]:
a_6 = '''
SELECT firstname, surname
FROM Members
WHERE joindate = (
    SELECT MAX(joindate) FROM Members
)
;
'''
print('\n\nmember(s) with the most recent join date')
q_6 = query_result(a_6, row_nums = False)



member(s) with the most recent join date

firstname    |  surname
_______________________________________________________________________________
Darren       |  Smith  
_______________________________________________________________________________
result: 1 rows, 2 columns




<a id = 'q7'></a>**Q 7**
<br>The strings assigned to variables `a_7`, `a_8`, and `a_9` represent the complete statements used to obtain desired results in MySQL from the database instance at https://sql.springboard.com/. To obtain the same results in this notebook, strings are sliced to remove the `SET sql_mode` statement before passing to `query_result()`.

In [10]:
a_7 = '''
SET sql_mode = PIPES_AS_CONCAT;
SELECT
    court_name||': '||firstname||' '||surname
        AS tennis_court_booking_str
FROM
(
    SELECT DISTINCT
        f.name AS court_name, 
        m.firstname,
        m.surname
    FROM 
        Bookings b
    JOIN 
        Facilities f
        ON b.facid = f.facid
    JOIN 
        Members m
        ON b.memid = m.memid
    WHERE
        UPPER(f.name) LIKE '%TENNIS%COURT%'
    AND m.memid != 0
) tennis_court_bookings
ORDER BY
    surname, firstname, court_name
;
'''
print('\n\nNames of all members who booked each tennis court:')
q_7 = query_result(a_7[a_7.index(';') + 1:])



Names of all members who booked each tennis court:

 #  |  tennis court booking str         
_______________________________________________________________________________
 0  |  Tennis Court 1: Florence Bader   
 1  |  Tennis Court 2: Florence Bader   
 2  |  Tennis Court 1: Anne Baker       
 3  |  Tennis Court 2: Anne Baker       
 4  |  Tennis Court 1: Timothy Baker    
 5  |  Tennis Court 2: Timothy Baker    
 6  |  Tennis Court 1: Tim Boothe       
 7  |  Tennis Court 2: Tim Boothe       
 8  |  Tennis Court 1: Gerald Butters   
 9  |  Tennis Court 2: Gerald Butters   
10  |  Tennis Court 1: Joan Coplin      
11  |  Tennis Court 1: Erica Crumpet    
12  |  Tennis Court 1: Nancy Dare       
13  |  Tennis Court 2: Nancy Dare       
14  |  Tennis Court 1: David Farrell    
15  |  Tennis Court 2: David Farrell    
16  |  Tennis Court 1: Jemima Farrell   
17  |  Tennis Court 2: Jemima Farrell   
18  |  Tennis Court 1: Matthew Genting  
19  |  Tennis Court 1: John Hunt        
20  |

<a id = 'q8'></a>**Q 8**

In [11]:
a_8 = '''
SET sql_mode = PIPES_AS_CONCAT;
SELECT
    f.name || ': ' || m.firstname || ' ' || m.surname 
        AS booking_info,
    CASE
        WHEN b.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
    END as booking_cost
FROM
    Bookings b
    JOIN Facilities f
        ON b.facid = f.facid
    JOIN Members m
        ON b.memid = m.memid
WHERE
    DATE(b.starttime) = DATE('2012-09-14') AND
    CASE
        WHEN b.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
    END  > 30
ORDER BY
    booking_cost DESC
;
'''
print('\n\nBookings for 2012-09-14 that cost more than $30:')
q_8 = query_result(a_8[a_8.index(';') + 1:])



Bookings for 2012-09-14 that cost more than $30:

 #  |  booking info                      |  booking cost
_______________________________________________________________________________
 0  |  Massage Room 2: GUEST GUEST       |  320         
 1  |  Massage Room 1: GUEST GUEST       |  160         
 2  |  Massage Room 1: GUEST GUEST       |  160         
 3  |  Massage Room 1: GUEST GUEST       |  160         
 4  |  Tennis Court 2: GUEST GUEST       |  150         
 5  |  Tennis Court 1: GUEST GUEST       |  75          
 6  |  Tennis Court 1: GUEST GUEST       |  75          
 7  |  Tennis Court 2: GUEST GUEST       |  75          
 8  |  Squash Court: GUEST GUEST         |  70.0        
 9  |  Massage Room 1: Jemima Farrell    |  39.6        
10  |  Squash Court: GUEST GUEST         |  35.0        
11  |  Squash Court: GUEST GUEST         |  35.0        
_______________________________________________________________________________
result: 12 rows, 2 columns




<a id = 'q9'></a>**Q 9**

In [12]:
a_9 = '''
SET sql_mode = PIPES_AS_CONCAT;
SELECT
    bc.facility_name || ': ' || m.firstname || ' ' || m.surname
        AS booking_info,
    bc.booking_cost
FROM
(
    SELECT
        f.name AS facility_name, 
        b.memid,
        CASE 
            WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
        END AS booking_cost
    FROM
        Bookings b
        JOIN Facilities f
            ON b.facid = f.facid
    WHERE
        b.starttime LIKE '2012-09-14%'
) bc
    JOIN Members m
        ON bc.memid = m.memid
WHERE
    bc.booking_cost > 30
ORDER BY
    bc.booking_cost DESC
;
'''
print('\n\nBookings for 2012-09-14 that cost more than $30:')
print('(using a subquery)')
q_9 = query_result(a_9[a_9.index(';') + 1:])



Bookings for 2012-09-14 that cost more than $30:
(using a subquery)

 #  |  booking info                      |  booking cost
_______________________________________________________________________________
 0  |  Massage Room 2: GUEST GUEST       |  320         
 1  |  Massage Room 1: GUEST GUEST       |  160         
 2  |  Massage Room 1: GUEST GUEST       |  160         
 3  |  Massage Room 1: GUEST GUEST       |  160         
 4  |  Tennis Court 2: GUEST GUEST       |  150         
 5  |  Tennis Court 1: GUEST GUEST       |  75          
 6  |  Tennis Court 1: GUEST GUEST       |  75          
 7  |  Tennis Court 2: GUEST GUEST       |  75          
 8  |  Squash Court: GUEST GUEST         |  70.0        
 9  |  Massage Room 1: Jemima Farrell    |  39.6        
10  |  Squash Court: GUEST GUEST         |  35.0        
11  |  Squash Court: GUEST GUEST         |  35.0        
_______________________________________________________________________________
result: 12 rows, 2 columns



### Questions 10-13

<a id = 'q10'></a>**Q 10**

In [13]:
a_10 = '''
WITH 
facility_booking AS
(
    SELECT
        f.name AS facility_name,
        b.bookid, 
        b.facid,
        CASE
            WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
        END AS booking_price
    FROM
        Bookings b JOIN Facilities f
            ON b.facid = f.facid
),
facility_revenue AS
(
    SELECT
        facid AS id,
        facility_name,
        SUM(booking_price) AS total_revenue
    FROM
        facility_booking
    GROUP BY
        facility_name, facid
)
SELECT * FROM facility_revenue
WHERE total_revenue < 1000
ORDER BY total_revenue
;
'''
print('\n\nfacilities with total revenue less than 1000')
q_10 = query_result(a_10, row_nums = False)



facilities with total revenue less than 1000

id    |  facility name    |  total revenue
_______________________________________________________________________________
3     |  Table Tennis     |  180          
7     |  Snooker Table    |  240          
8     |  Pool Table       |  270          
_______________________________________________________________________________
result: 3 rows, 3 columns




<a id = 'q11'></a>**Q 11**

In [14]:
a_11 = '''
SELECT
    m.firstname||' '||m.surname AS member_name,
    IFNULL(r.firstname||' '||r.surname, '') AS recommended_by
FROM
    Members m
    LEFT JOIN Members r 
        on m.recommendedby = r.memid
WHERE
    m.memid != 0
ORDER BY
    m.surname, m.firstname, r.surname, r.firstname
;
'''
print('\n\ncountry club members\nwith recommending member where applicable')
q_11 = query_result(a_11)



country club members
with recommending member where applicable

 #  |  member name                |  recommended by   
_______________________________________________________________________________
 0  |  Florence Bader             |  Ponder Stibbons  
 1  |  Anne Baker                 |  Ponder Stibbons  
 2  |  Timothy Baker              |  Jemima Farrell   
 3  |  Tim Boothe                 |  Tim Rownam       
 4  |  Gerald Butters             |  Darren Smith     
 5  |  Joan Coplin                |  Timothy Baker    
 6  |  Erica Crumpet              |  Tracy Smith      
 7  |  Nancy Dare                 |  Janice Joplette  
 8  |  David Farrell              |                   
 9  |  Jemima Farrell             |                   
10  |  Matthew Genting            |  Gerald Butters   
11  |  John Hunt                  |  Millicent Purview
12  |  David Jones                |  Janice Joplette  
13  |  Douglas Jones              |  David Jones      
14  |  Janice Joplette       

<a id = 'q12'></a>**Q 12**

I found question 12 difficult to understand. The question reads: <br>
"`/* Q12: Find the facilities with their usage by member, but not guests */`" (line 92)<br>
<br> The query assigned to `a_12_i` returns a list of facilities that were booked at least once by a member but never booked by a guest. The query assigned to `a_12_ii` returns a list of summary statistics based on the bookings made by members for each facility.

In [15]:
a_12_i = '''
SELECT DISTINCT
    b.facid, f.name
FROM 
    Bookings b 
    JOIN Facilities f
        ON b.facid = f.facid
WHERE 
    b.memid != 0 AND
    b.facid NOT IN (
        SELECT DISTINCT facid 
        FROM Bookings 
        WHERE memid = 0
    )
;
'''
print('facilities booked by members but not booked by guests')
q_12_i = query_result(a_12_i, row_nums = False)


facilities booked by members but not booked by guests

facid    |  name
_______________________________________________________________________________
_______________________________________________________________________________
result: 0 rows, 2 columns




In [16]:
a_12_ii = '''
WITH
member_usage_summary AS
(
    SELECT 
        facid, 
        COUNT(DISTINCT memid) AS members,
        COUNT(DISTINCT bookid) AS bookings,
        SUM(slots) AS total_slots
    FROM Bookings
    WHERE memid != 0
    GROUP BY facid
)
SELECT
    f.facid AS id,
    f.name AS facility_name,
    IFNULL(m.members, 0) AS total_members,
    IFNULL(m.bookings, 0) AS total_bookings,
    IFNULL(m.total_slots, 0) AS total_slots_booked
FROM
    Facilities f 
    LEFT JOIN member_usage_summary m
        on f.facid = m.facid
    
'''
print('\n\n2012 facility usage by members:')
q_12_ii = query_result(a_12_ii, row_nums = False, pg_w = 100)



2012 facility usage by members:

id    |  facility name      |  total members    |  total bookings    |  total slots booked
____________________________________________________________________________________________________
0     |  Tennis Court 1     |  23               |  308               |  957               
1     |  Tennis Court 2     |  21               |  276               |  882               
2     |  Badminton Court    |  24               |  344               |  1086              
3     |  Table Tennis       |  25               |  385               |  794               
4     |  Massage Room 1     |  24               |  421               |  884               
5     |  Massage Room 2     |  12               |  27                |  54                
6     |  Squash Court       |  24               |  195               |  418               
7     |  Snooker Table      |  22               |  421               |  860               
8     |  Pool Table         |  27            

<a id = 'q13'></a>**Q 13** 

As before, I found the phrasing of the question somewhat awkward. It reads:<br>
"`/* Q13: Find the facilities usage by month, but not guests */`" (line 95) <br>
<br>
My solutions to this question mirror those of Q12.

To start, get distinct year-month pairs from the Bookings table.

In [17]:
yyyy_and_mm = '''
SELECT DISTINCT 
    SUBSTR(DATE(starttime), 1, 4) AS year,
    SUBSTR(DATE(starttime), 6, 2) AS month
FROM
    Bookings
;
'''
yyyy_and_mm = query_result(yyyy_and_mm, row_nums = False)



year    |  month
_______________________________________________________________________________
2012    |  07   
2012    |  08   
2012    |  09   
_______________________________________________________________________________
result: 3 rows, 2 columns




There is only one year in the dataset, and there are only three months. This simplifies grouping and pivoting data.<br><br> `a_13_i` returns a list of facilities booked by at least one member in a month that were not booked by a guest during that month.


In [18]:
a_13_i = '''
WITH bookings_by_mm AS
(
    SELECT DISTINCT 
        facid, 
        SUBSTR(starttime, 6, 2) AS mm,
        CASE 
            WHEN memid = 0 THEN 0
            ELSE 1
        END AS is_member
    FROM
        Bookings
)
SELECT
    b.facid, 
    f.name AS facility_name,
    b.mm AS month 
FROM
    bookings_by_mm b
    JOIN Facilities f
        ON b.facid = f.facid
WHERE
    b.is_member = 1 AND b.facid NOT IN
    (
        SELECT 
            g.facid 
        FROM
            bookings_by_mm g
        WHERE 
            g.facid = b.facid
            AND g.mm = b.mm
            AND g.is_member = 0
    )

'''
q_13_i = query_result(a_13_i)


facid    |  facility name    |  month
_______________________________________________________________________________
_______________________________________________________________________________
result: 0 rows, 3 columns




For part ii, generate a list of months using the date information collected above. Use this list to write an extended series of LEFT JOINs as a functional replacement for PIVOT. Use f-strings to build query components.

In [19]:
months = [
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December'
]
months_dict = {row[1]: months[int(row[1].lstrip('0')) - 1] for row in yyyy_and_mm[1]}
months = [month for month in months_dict.values()]
months_str_a = [f'\'{key}\' AS {val}' for key, val in months_dict.items()]
months_str_b = [f'mb{i + 1}' for i in range(len(months))]
months_str_c = '\n'.join(
    f'LEFT JOIN monthly_bookings {months_str_b[i]}\n\t' 
    f'ON fm.facid = {months_str_b[i]}.facid AND\n\t'
    f'fm.{months[i]} = {months_str_b[i]}.month'
    for i in range(len(months))
)
months_str_a = ', '.join(months_str_a)
print(months)
print(', '.join(f'{b}.month' for b in months_str_b))
print(f'{months_str_a}\n{months_str_c}')

['July', 'August', 'September']
mb1.month, mb2.month, mb3.month
'07' AS July, '08' AS August, '09' AS September
LEFT JOIN monthly_bookings mb1
	ON fm.facid = mb1.facid AND
	fm.July = mb1.month
LEFT JOIN monthly_bookings mb2
	ON fm.facid = mb2.facid AND
	fm.August = mb2.month
LEFT JOIN monthly_bookings mb3
	ON fm.facid = mb3.facid AND
	fm.September = mb3.month


Use components to construct queries generating summary statistics by month.

In [20]:
a_13_pt_1 = '''
WITH
facilities_months AS
(
    SELECT DISTINCT 
        facid, name,
        '07' AS July, '08' AS August, '09' AS September
    FROM Facilities
),
monthly_bookings AS
(
    SELECT
        facid,
        SUBSTR(starttime, 6, 2) AS month,
        SUM(slots) AS booked_slots,
        COUNT(DISTINCT memid) AS distinct_users,
        COUNT(DISTINCT bookid) AS total_bookings
    FROM
        Bookings
    WHERE
        memid != 0
    GROUP BY
        facid,
        SUBSTR(starttime, 6, 2)
)
'''
a_13_pt_2 = '''
SELECT
    fm.facid as id, 
    fm.name,
'''
a_13_pt_3 = [
    ''.join([
        f'IFNULL(mb1.{col}, 0) AS July, \n',
        f'IFNULL(mb2.{col}, 0) AS August, \n',
        f'IFNULL(mb3.{col}, 0) AS September \n'
    ]) for col in ['booked_slots', 'distinct_users', 'total_bookings']
]
a_13_pt_4 = '''
FROM
    facilities_months fm
LEFT JOIN monthly_bookings mb1
    ON fm.facid = mb1.facid AND
    fm.July = mb1.month
LEFT JOIN monthly_bookings mb2
    ON fm.facid = mb2.facid AND
    fm.August = mb2.month
LEFT JOIN monthly_bookings mb3
    ON fm.facid = mb3.facid AND
    fm.September = mb3.month
;
'''
a_13_ii = [
    ''.join([a_13_pt_1, a_13_pt_2, a_13_pt_3[i], a_13_pt_4]) 
    for i in range(3)]
q_13_booked_slots, q_13_distinct_users, q_13_total_bookings = (
    query_result(a_13_ii[i], print_result = False)
    for i in range(3)
)
q_13_ii = {
    'total slots booked': q_13_booked_slots,
    'total members' : q_13_distinct_users, 
    'total bookings': q_13_total_bookings
}

Print the queries passed to `query_result`.

In [21]:
for query in a_13_ii:
    print(query)


WITH
facilities_months AS
(
    SELECT DISTINCT 
        facid, name,
        '07' AS July, '08' AS August, '09' AS September
    FROM Facilities
),
monthly_bookings AS
(
    SELECT
        facid,
        SUBSTR(starttime, 6, 2) AS month,
        SUM(slots) AS booked_slots,
        COUNT(DISTINCT memid) AS distinct_users,
        COUNT(DISTINCT bookid) AS total_bookings
    FROM
        Bookings
    WHERE
        memid != 0
    GROUP BY
        facid,
        SUBSTR(starttime, 6, 2)
)

SELECT
    fm.facid as id, 
    fm.name,
IFNULL(mb1.booked_slots, 0) AS July, 
IFNULL(mb2.booked_slots, 0) AS August, 
IFNULL(mb3.booked_slots, 0) AS September 

FROM
    facilities_months fm
LEFT JOIN monthly_bookings mb1
    ON fm.facid = mb1.facid AND
    fm.July = mb1.month
LEFT JOIN monthly_bookings mb2
    ON fm.facid = mb2.facid AND
    fm.August = mb2.month
LEFT JOIN monthly_bookings mb3
    ON fm.facid = mb3.facid AND
    fm.September = mb3.month
;


WITH
facilities_months AS
(
    SELECT DISTI

Print `q_13_ii` results.

In [22]:
for title, results in q_13_ii.items():
    print(f'\n\n2012 facilities usage by members:\n{title} by month')
    query_print(results[0], results[1])
    



2012 facilities usage by members:
total slots booked by month

#  |  id    |  name               |  July    |  August    |  September
_______________________________________________________________________________
0  |  0     |  Tennis Court 1     |  201     |  339       |  417      
1  |  1     |  Tennis Court 2     |  123     |  345       |  414      
2  |  2     |  Badminton Court    |  165     |  414       |  507      
3  |  3     |  Table Tennis       |  98      |  296       |  400      
4  |  4     |  Massage Room 1     |  166     |  316       |  402      
5  |  5     |  Massage Room 2     |  8       |  18        |  28       
6  |  6     |  Squash Court       |  50      |  184       |  184      
7  |  7     |  Snooker Table      |  140     |  316       |  404      
8  |  8     |  Pool Table         |  110     |  303       |  443      
_______________________________________________________________________________
result: 9 rows, 5 columns




2012 facilities usage by members:
t